<img src="https://microsoft.github.io/Accera/assets/Accera_darktext.png" alt="Accera logo" width="600"/>

# Accera Quickstart

### Install Accera

First, we'll install Accera using `pip`.

#### Requirements for running locally

To run this notebook locally:

* Linux/macOS: install gcc using `apt install gcc`.
* Windows: install Microsoft Visual Studio and run `vcvars64.bat` to setup Visual Studio tools in your `PATH` before starting the Jupyter environment.

In [ ]:
!pip install accera==1.2.1.dev42

### Build your first package

We'll build a package called `"mypackage"`, containing one function called `"hello_accera"`.

This function performs the operation `B += A` on arrays `A` and `B`.

In [ ]:
import accera as acc
import hatlib as hat
import numpy as np

A = acc.Array(role=acc.Array.Role.INPUT, shape=(16, 16))
B = acc.Array(role=acc.Array.Role.INPUT_OUTPUT, shape=(16, 16))

nest = acc.Nest(shape=(16, 16))
i, j = nest.get_indices()

@nest.iteration_logic
def _():
    B[i, j] += A[i, j]

package = acc.Package()
package.add(nest, args=(A, B), base_name="hello_accera")
package.build(name="mypackage", format=acc.Package.Format.HAT_DYNAMIC)

### Load the package and call the function

`package.build` produces a dynamic library (`mypackage_*.so`) that exports the `hello_accera` function. 

Let's call our function with some NumPy arrays.

In [ ]:
hat_package = hat.load("mypackage.hat")
hello_accera = hat_package["hello_accera"]

# call the function with test input
A_test = np.ones((16, 16), dtype=np.float32) * 3
B_test = np.ones((16, 16), dtype=np.float32) * .14
print(f"Before hello_accera(): {B_test}")
hello_accera(A_test, B_test)
print(f"After hello_accera(): {B_test}")